In [ ]:
import cv2
import mediapipe as mp
import pandas as pd
import re
from moviepy.editor import *
import time
from PIL import Image

import numpy as np



mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5, static_image_mode=True)
mp_holistic = mp.solutions.holistic

df = pd.DataFrame()
frames = []

In [ ]:
# BaseOptions = mp.tasks.BaseOptions
# HandLandmarker = mp.tasks.vision.HandLandmarker
# HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
# VisionRunningMode = mp.tasks.vision.RunningMode


# options = HandLandmarkerOptions(
#     base_options=BaseOptions(model_asset_path='D:\Documents\Trabajo\megaproyecto_model\hand_landmarker.task'),
#     running_mode=VisionRunningMode.IMAGE
# )
# HandLandmarker.create_from_model_path("D:\Documents\Trabajo\megaproyecto_model\hand_landmarker.task")

In [ ]:
import os
import copy
path = os.getcwd()

In [ ]:
import math
import matplotlib.pyplot as plt

def normalize_coordinates(coordinates):
    max_x = max(coord[0] for coord in coordinates)
    max_y = max(coord[1] for coord in coordinates)
    
    normalized_coordinates = []
    for x, y in coordinates:
        normalized_x = x / max_x
        normalized_y = y / max_y
        normalized_coordinates.append([normalized_x, normalized_y])
    
    return normalized_coordinates

def visualize_data(normalized_coordinates):
    # Unzip normalized coordinates for plotting
    normalized_x, normalized_y = zip(*normalized_coordinates)
    
    # Plot the normalized coordinates
    plt.figure(figsize=(8, 6))
    plt.scatter(normalized_x, normalized_y, color='blue', label='Normalized Coordinates')
    plt.xlabel('Normalized X')
    plt.xlabel('Normalized X')
    plt.ylabel('Normalized Y')
    plt.title('Normalized Coordinates Plot')
    plt.legend()
    plt.grid(True)
    plt.show()


In [ ]:
def generate_centered_image(detected_pixels):
    normalized_coordinates = normalize_coordinates(detected_pixels)
    # visualize_data(normalized_coordinates)

    return normalized_coordinates

In [ ]:
def extract_video(name, target, sequence_id):
    global frames
    detected_pixels = []

    # Load an image from file

    image = cv2.imdecode(np.fromfile(name, np.uint8), cv2.IMREAD_UNCHANGED)
    # Check if the image was loaded successfully
    if image is not None:
        # Convert the BGR frame to RGB
        rgb_frame = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Process the frame with the hand tracking model
        results = hands.process(rgb_frame)
         
        if results.multi_hand_landmarks:
            row_data = {
                "sequence_id": sequence_id,
                "target": target
            }
            hand_side = "Left"
            detected_hands = []
            for idx, landmarks in enumerate(results.multi_hand_landmarks):
                # Iterate through detected hand landmarks
                for landmark_idx, landmark in enumerate(landmarks.landmark):
                    x, y = landmark.x * image.shape[1], landmark.y * image.shape[0]
                    detected_pixels.append([x, y])

                    # Draw circles on the frame
                    # cv2.circle(image, (int(x), int(y)), 5, (0, 255, 0), -1)
                    cv2.putText(image, hand_side, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                    mp_drawing.draw_landmarks(image, landmarks, mp_hands.HAND_CONNECTIONS)

                break  
            normalized_coordinates_relative = generate_centered_image(detected_pixels)
            if (len(normalized_coordinates_relative) > 21):
                print(target, sequence_id, normalized_coordinates_relative)
            for i in range(len(normalized_coordinates_relative)):
                x, y = normalized_coordinates_relative[i]
                row_data[f'x_{hand_side}_hand_{i}'] =  x
                row_data[f'y_{hand_side}_hand_{i}'] =  y

            frames.append(row_data)
        
        cv2.imshow('Original', image)
        
    else:
        print("Failed to load image.")

    cv2.waitKey(0)
    # cv2.destroyAllWindows()

In [ ]:
import glob, os
import time
os.chdir("D:\Documents\Trabajo\megaproyecto_model\data/asl_letters_lensegua")

In [ ]:
for file in glob.glob("*"):
    if (file.split(".")[-1] != "png"):
        splitted_name = file.split(".")[:-1]
        splitted_name.append(".png")
        renamed_format = "".join(splitted_name)

        im = Image.open(file)
        im.save(renamed_format)

        os.remove(file)


In [ ]:
os.chdir("D:\Documents\Trabajo\megaproyecto_model\data/asl_letters_lensegua")

sequence_id = 0
for file in glob.glob("*.png"):
    sequence_id += 1
    name = " ".join(file.split(" ")[:-1])

    extract_video(file, name, sequence_id)

In [ ]:
df = pd.DataFrame(frames)

In [ ]:
df['sequence_id'] = df['sequence_id'].astype(int)

In [ ]:
len(df)

In [ ]:
df.head()

In [ ]:
dir_data = "D:\Documents\Trabajo\megaproyecto_model/data" + "/data_letters.csv"
df.to_csv(dir_data, index=False)

In [ ]:
len(df.columns)

In [ ]:
validation_path = "D:\Documents\Trabajo\megaproyecto_model/data" + "/validation_letters.csv"

import random
grouped = df.groupby('target')
selected_validation_ids = []
for group_name, group_indices in grouped.groups.items():
    ids = list(set(df.loc[group_indices]['sequence_id'].values))
    if (len(ids) < 0):
        raise ("Error")
    selected_validation_ids.append(random.choice(ids))

validation_df = df[df['sequence_id'].isin(selected_validation_ids)][["sequence_id", "target"]]
validation_df.to_csv(validation_path, index=False)

In [ ]:
train_path = "D:\Documents\Trabajo\megaproyecto_model/data" + "/train_letters.csv"
train_df = df[~df['sequence_id'].isin(selected_validation_ids)][["sequence_id", "target"]]
train_df.to_csv(train_path, index=False)